In [ ]:
import subprocess
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as π
import tqdm
import pygmsh
import firedrake
from firedrake import (
    inner,
    grad,
    dx,
    ds,
    derivative,
    NonlinearVariationalProblem,
    NonlinearVariationalSolver,
)
import irksome
from icepack2 import model, solvers
from icepack2.constants import glen_flow_law as n

Create the initial geometry.

In [ ]:
R = 200e3
δx = 5e3

geometry = pygmsh.built_in.Geometry()

x1 = geometry.add_point([-R, 0, 0], lcar=δx)
x2 = geometry.add_point([+R, 0, 0], lcar=δx)

center1 = geometry.add_point([0, 0, 0], lcar=δx)
center2 = geometry.add_point([0, -4 * R, 0], lcar=δx)

arcs = [
    geometry.add_circle_arc(x1, center1, x2),
    geometry.add_circle_arc(x2, center2, x1),
]

line_loop = geometry.add_line_loop(arcs)
plane_surface = geometry.add_plane_surface(line_loop)

physical_lines = [geometry.add_physical(arc) for arc in arcs]
physical_surface = geometry.add_physical(plane_surface)

with open("ice-shelf.geo", "w") as geo_file:
    geo_file.write(geometry.get_code())

command = "gmsh -2 -format msh2 -v 0 -o ice-shelf.msh ice-shelf.geo"
subprocess.run(command.split())

mesh = firedrake.Mesh("ice-shelf.msh")

Make the initial thickness and velocity fields.

In [ ]:
inlet_angles = π * np.array([-3 / 4, -1 / 2, -1 / 3, -1 / 6])
inlet_widths = π * np.array([1 / 8, 1 / 12, 1 / 24, 1 / 12])

x = firedrake.SpatialCoordinate(mesh)

u_in = 300
h_in = 350
hb = 100
dh, du = 400, 250

hs, us = [], []
for θ, ϕ in zip(inlet_angles, inlet_widths):
    x0 = R * firedrake.as_vector((np.cos(θ), np.sin(θ)))
    v = -firedrake.as_vector((np.cos(θ), np.sin(θ)))
    L = inner(x - x0, v)
    W = x - x0 - L * v
    Rn = 2 * ϕ / π * R
    q = firedrake.max_value(1 - (W / Rn) ** 2, 0)
    hs.append(hb + q * ((h_in - hb) - dh * L / R))
    us.append(firedrake.exp(-4 * (W / R) ** 2) * (u_in + du * L / R) * v)

h_expr = firedrake.Constant(hb)
for h in hs:
    h_expr = firedrake.max_value(h, h_expr)

u_expr = sum(us)

Make some function spaces and initialize the solution.

In [ ]:
cg = firedrake.FiniteElement("CG", "triangle", 1)
dg0 = firedrake.FiniteElement("DG", "triangle", 0)
dg1 = firedrake.FiniteElement("DG", "triangle", 1)
Q = firedrake.FunctionSpace(mesh, dg1)
V = firedrake.VectorFunctionSpace(mesh, cg)
Σ = firedrake.TensorFunctionSpace(mesh, dg0, symmetry=True)
Z = V * Σ

h0 = firedrake.Function(Q).interpolate(h_expr)
u0 = firedrake.Function(V).interpolate(u_expr)

h = h0.copy(deepcopy=True)
z = firedrake.Function(Z)
z.sub(0).assign(u0);

Set up the momentum balance equation.

In [ ]:
ε_c = firedrake.Constant(0.01)
τ_c = firedrake.Constant(0.1)

u, M = firedrake.split(z)
fields = {
    "velocity": u,
    "membrane_stress": M,
    "thickness": h,
}

fns = [model.viscous_power, model.ice_shelf_momentum_balance]

rheology = {
    "flow_law_exponent": n,
    "flow_law_coefficient": ε_c / τ_c ** n,
}

L = sum(fn(**fields, **rheology) for fn in fns)
F = derivative(L, z)

Make an initial guess by solving a Picard linearization.

In [ ]:
linear_rheology = {
    "flow_law_exponent": 1,
    "flow_law_coefficient": ε_c / τ_c,
}

L_1 = sum(fn(**fields, **linear_rheology) for fn in fns)
F_1 = derivative(L_1, z)

qdegree = n + 2
bc = firedrake.DirichletBC(Z.sub(0), u0, (1,))
problem_params = {
    #"form_compiler_parameters": {"quadrature_degree": qdegree},
    "bcs": bc,
}
solver_params = {
    "solver_parameters": {
        "snes_type": "newtonls",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
        "snes_rtol": 1e-2,
    },
}
firedrake.solve(F_1 == 0, z, **problem_params, **solver_params)

Now solve the real problem but using a perturbed linearization.

In [ ]:
h_min = firedrake.Constant(1.0)
rfields = {
    "velocity": u,
    "membrane_stress": M,
    "thickness": firedrake.max_value(h_min, h),
}

L_r = sum(fn(**rfields, **rheology) for fn in fns)
F_r = derivative(L_r, z)
H_r = derivative(F_r, z)

In [ ]:
bc = firedrake.DirichletBC(Z.sub(0), firedrake.Constant((0, 0)), (1,))
problem = solvers.ConstrainedOptimizationProblem(L, z, H=H_r, bcs=bc)
diagnostic_solver = solvers.NewtonSolver(problem)
residuals = [diagnostic_solver.solve()]

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.quiver(z.subfunctions[0], axes=ax)
fig.colorbar(colors);

### Simulation

Now add the mass balance part of the problem.

In [ ]:
prognostic_problem = model.mass_balance(
    thickness=h,
    velocity=u,
    accumulation=firedrake.Constant(0.0),
    thickness_inflow=h0,
    test_function=firedrake.TestFunction(Q),
)

final_time = 400.0
num_steps = 400

dt = firedrake.Constant(final_time / num_steps)
t = firedrake.Constant(0.0)
method = irksome.BackwardEuler()
prognostic_params = {
    "solver_parameters": {
        "snes_type": "ksponly",
        "ksp_type": "gmres",
        "pc_type": "bjacobi",
    },
}
prognostic_solver = irksome.TimeStepper(
    prognostic_problem, method, t, dt, h, **prognostic_params
)

In [ ]:
for step in tqdm.trange(num_steps):
    prognostic_solver.advance()
    residuals.append(diagnostic_solver.solve())

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors);

In [ ]:
fig, ax = plt.subplots()
ax.plot([len(r) for r in residuals]);

In [ ]:
u = firedrake.Function(V)
M = firedrake.Function(Σ)
u.assign(z.subfunctions[0])
M.assign(z.subfunctions[1])

with firedrake.CheckpointFile("gibbous.h5", "w") as chk:
    chk.save_mesh(mesh)
    chk.save_function(h, name="thickness")
    chk.save_function(u, name="velocity")
    chk.save_function(M, name="stress")